### I'm testing with a corpus . . . 

 . . . with one file.  I test all of this less the line number logic using another, larger corpus. 

In [1]:
import glob

paths_to_files = glob.glob('/home/spenteco/Downloads/test_corpus/*.txt')

print(len(paths_to_files))

# I don't know whether these are place names or not.  Let's pretend they are.
terms_to_find = [r'du\s*Dépôt\s*de\s*mendicité\s*de\s*Saint-Denis', 'Saint-Gervais']

1


## A routine for writing XML

We call this from below . . . 

In [2]:
from lxml import etree

def write_results_to_xml(file_name, results):

    xml_root = etree.Element('matches_and_snippets')
    
    for r in results:
        
        match_node = etree.Element('match')
        xml_root.append(match_node)
        
        file_name_node = etree.Element('file_name')
        file_name_node.text = r['file_name']
        match_node.append(file_name_node)
        
        pdf_page_number_node = etree.Element('pdf_page_number')
        pdf_page_number_node.text = str(r['pdf_page_number'])
        match_node.append(pdf_page_number_node)
        match_node.append(file_name_node)
        
        book_page_number_node = etree.Element('book_page_number')
        book_page_number_node.text = ''
        match_node.append(book_page_number_node)
        
        matching_term_node = etree.Element('matching_term')
        matching_term_node.text = r['matching_term']
        match_node.append(matching_term_node)
        
        snippet_node = etree.Element('snippet')
        snippet_node.text = r['snippet']
        match_node.append(snippet_node)
        
    xml_root.getroottree().write(file_name.replace('.txt', '.xml'), encoding='utf-8', pretty_print=True)

### Using (very straight-forward) regular expressions

The regular expression can be just a string, like "Saint-Gervais".

Something like 

> r'du\s*Dépôt\s*de\s*mendicité\s*de\s*Saint-Denis' 
    
lets us search across line breaks, even when the OCR has inserted extra line breaks.  Note that the leading r matters.

In [3]:
import re

N_CHARACTERS_BEFORE_MATCH = 1000
N_CHARACTERS_AFTER_MATCH = 2000

for path_to_file in paths_to_files:

    results = []
    
    # Get the file name less the path; we use the full path to read, and this file name in the printed output
    file_name = path_to_file.split('/')[-1]
    
    text = open(path_to_file).read()
    
    # Where are the form-feed characters in the document?
    
    page_break_locations = []
    
    for match in re.finditer('\x0c', text):
        page_break_locations.append(match.start())
    
    # The actual term matching
    
    for term in terms_to_find:
        
        for match in re.finditer(term, text):
            
            pdf_page_number = 1
            for location in page_break_locations:
                if location > match.start():
                    break
                pdf_page_number += 1
            
            snippet_starting_position = match.start() - N_CHARACTERS_BEFORE_MATCH
            if snippet_starting_position < 0:
                snippet_starting_position = 0
                
            snippet_ending_position = match.end() + N_CHARACTERS_AFTER_MATCH
            if snippet_ending_position > len(text):
                snippet_ending_position = len(text)
                
            snippet = text[snippet_starting_position: snippet_ending_position]
            
            results.append({'file_name': file_name, 
                            'pdf_page_number': pdf_page_number,
                            'matching_term': text[match.start(): match.end()], 
                            'snippet': snippet.replace('\x0c', '')})
            
    if len(results) > 0:
        
        write_results_to_xml(file_name, results)
        
print('Done!')

Done!
